## Project

In [2]:
import pandas as pd
import numpy as np
import requests
import os
import xarray as xr
from netCDF4 import Dataset
import tempfile
from scipy.spatial.distance import cdist
from tqdm import tqdm
from math import radians, cos, sin, asin, sqrt

### 1. Data Collection

#### 1.1 Biomass data

Spatio-temporal data for walleye pollock in the Gulf of Alaska extracted from the NOAA Fisheries website: https://apps-st.fisheries.noaa.gov/dismap/DisMAP.html#single-species-distributions

In [3]:
wp_gulf_alaska = pd.read_csv("wp_gulf_alaska.csv")
wp_gulf_alaska.head()

,HaulID,Stratum,LAT,LON,Depth,Year,wtcpue
0,021-198403-001,350,55.98767,-134.59517,346,1984,0.0000
1,021-198403-002,251,55.63200,-134.22900,225,1984,4.7482
2,021-198403-003,151,55.21450,-133.88733,117,1984,1.0413
3,021-198403-006,151,55.13300,-133.87100,141,1984,1.3450
4,021-198403-007,151,55.12550,-134.00167,170,1984,0.2030


Columns:
- HaulID / Stratum: characterizes the survey expedition and catch throw.
- LAT / LON: Latitude and Longitude of survey catch
- Depth: Sea depth measured in meters below sea level.
- Year: No other timestamp is provided. However, documentation specify that all surveys have been conducted during summer months.
- wtcpue: survey catch measured in kg per ha.


In [4]:
wp_gulf_alaska = wp_gulf_alaska.rename(columns={"HaulID": "haul_id", "Stratum": "stratum", "LAT": "lat", "LON": "lon", "Depth":'depth', "Year": "year"})
wp_gulf_alaska['year'] = wp_gulf_alaska['year'].astype('int64')
wp_gulf_alaska.head()

,haul_id,stratum,lat,lon,depth,year,wtcpue
0,021-198403-001,350,55.98767,-134.59517,346,1984,0.0000
1,021-198403-002,251,55.63200,-134.22900,225,1984,4.7482
2,021-198403-003,151,55.21450,-133.88733,117,1984,1.0413
3,021-198403-006,151,55.13300,-133.87100,141,1984,1.3450
4,021-198403-007,151,55.12550,-134.00167,170,1984,0.2030


#### 1.2 Oceanographic data

We web-scrape temperature and salinity geospatial data from HyCOM (Hybrid Coordinate Ocean Model) using their NCSS request URL. Since there is too much data, we have decided to only get a sample from the first day of each summer month (July 1st, August 1st and September 1st). Of course, including more points should improve our model. Then again, we don't have a timestamp associated with each survey, just the year.

We will extract data from this geographic region:
- North limit: 60.32
- South limit: 52.41
- West limit: -170
- East limit: -132.5

Unfortunately, we do not have data prior to 1995. 

In [5]:
# Define the list of years and months
year_list = [1996, 1999, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]
month_list = [7, 8, 9]
this_folder = os.getcwd()
data_folder = this_folder + '\\temp'

if not os.path.exists('ocean_data.csv'):
  # create data folder if it doesn't exists 
  if not os.path.exists(data_folder):
    os.mkdir(data_folder)
  
  # Create an empty DataFrame to store the results
  df_all = pd.DataFrame()

  # Loop over each desired year and month, and add a progress bar
  for year in year_list:
      for month in tqdm(month_list, desc=f'{year}'):
          # Define the date string
          date_str = f'{year}-{month:02d}-01T09:00:00Z'
          # Define the URL with the fixed parameters
          if year <=2015:
              url = "https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_53.X/data/"+str(year)+"?var=salinity_bottom&var=water_temp_bottom&north=60.32&west=-170&east=-132.5&south=52.41&horizStride=1&vertCoord=&accept=netcdf4"
          elif year == 2017:
              url = "https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_57.7?var=salinity_bottom&var=water_temp_bottom&north=60.32&west=-170&east=-132.5&south=52.41&horizStride=1&vertCoord=&accept=netcdf4"
          else:
              url = "https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0/ts3z?var=salinity_bottom&var=water_temp_bottom&north=60.32&west=-170&east=-132.5&south=52.41&horizStride=1&vertCoord=&accept=netcdf4"
          
          my_file = os.path.join(this_folder, 'temp', f'example-{date_str[:10]}.nc4')
          # Add the date parameter to the URL
          url_day = f'{url}&time={date_str}'
          # Download the data and save it to a file
          downloaded_obj = requests.get(url_day)
          with open(my_file, "wb") as file:
              file.write(downloaded_obj.content)
              del downloaded_obj
          # Open the NetCDF file and convert it to a pandas DataFrame
          ds = xr.open_dataset(my_file)
          df_temp = ds.to_dataframe().reset_index()
          # Append the DataFrame to the overall DataFrame
          df_all = pd.concat([df_all, df_temp], ignore_index=True)

  df_all.to_csv('ocean_data.csv', index=False)

In [6]:
ocean_data_df = pd.read_csv('ocean_data.csv')
ocean_data_df.head()

,time,lat,lon,salinity_bottom,water_temp_bottom
0,1996-07-01 09:00:00,52.400002,-170.000000,34.223000,3.535999
1,1996-07-01 09:00:00,52.400002,-169.919983,34.287003,3.223999
2,1996-07-01 09:00:00,52.400002,-169.839966,34.330000,3.050999
3,1996-07-01 09:00:00,52.400002,-169.760010,34.323000,3.139000
4,1996-07-01 09:00:00,52.400002,-169.679993,34.324000,3.146000


In [7]:
ocean_data_df['time'] = pd.to_datetime(ocean_data_df['time'])
ocean_data_df['year'] = ocean_data_df['time'].dt.strftime('%Y').astype(int)
ocean_data_df.head()

,time,lat,lon,salinity_bottom,water_temp_bottom,year
0,1996-07-01 09:00:00,52.400002,-170.000000,34.223000,3.535999,1996
1,1996-07-01 09:00:00,52.400002,-169.919983,34.287003,3.223999,1996
2,1996-07-01 09:00:00,52.400002,-169.839966,34.330000,3.050999,1996
3,1996-07-01 09:00:00,52.400002,-169.760010,34.323000,3.139000,1996
4,1996-07-01 09:00:00,52.400002,-169.679993,34.324000,3.146000,1996


In [8]:
# Attempt at graphing wp_gulf_alaska to figure out how to trim down the ocean_data_df
# from shapely.geometry import Point
# import geopandas as gpd
# from geopandas import GeoDataFrame

# geometry = [Point(xy) for xy in zip(wp_gulf_alaska['lon'], wp_gulf_alaska['lat'])]
# gdf = GeoDataFrame(wp_gulf_alaska, geometry=geometry)   

# #this is a simple map that goes with geopandas
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

In [10]:
# df_mean = (ocean_data_df
#            .groupby(['lat', 'lon', 'year'])
#            [['salinity_bottom', 'water_temp_bottom']]
#            .mean()
#            .reset_index())
# df_mean['Year'] = df_mean['year'].astype('int64')
# df_mean.head()

#### 1.3 Merging the two data sets

In [11]:
sub_ocean_data_df = ocean_data_df.sample(10000)
sub_ocean_data_df['ocean_data_id'] = sub_ocean_data_df.index 
sub_ocean_data_df.head()
ocean_data_df.shape

(3086490, 6)

In [12]:
wp_gulf_alaska = wp_gulf_alaska[wp_gulf_alaska['year'].isin(ocean_data_df.year.unique())]
sub_wp_gulf_alaska = wp_gulf_alaska
sub_wp_gulf_alaska.head()

,haul_id,stratum,lat,lon,depth,year,wtcpue
3104,023-199601-003,210,52.59615,-169.4421,235,1996,9.0793
3105,023-199601-004,111,52.78445,-168.7133,109,1996,0.3296
3106,023-199601-005,10,52.85188,-168.6160,95,1996,646.7679
3107,023-199601-006,10,52.98299,-168.2722,106,1996,256.1357
3108,023-199601-007,210,52.96844,-167.5347,227,1996,3.0989


In [13]:
def dist(lat1, lon1, lat2, lon2):
    """
    https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546
    """
    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km

def find_nearest(lat, lon, year):
  year_ocean_data_df = sub_ocean_data_df[sub_ocean_data_df['year'] == year]
  
  # mini filtering on lat lng
  # year_ocean_data_df = year_ocean_data_df[(abs(year_ocean_data_df["lat"] - lat) < 10) & (abs(year_ocean_data_df["lon"] - lon) < 10)]
  
  distances = year_ocean_data_df.apply(
      lambda row: dist(lat, lon, row['lat'], row['lon']), 
      axis=1)
  return year_ocean_data_df.loc[distances.idxmin(), 'ocean_data_id']

sub_wp_gulf_alaska['ocean_data_id'] = sub_wp_gulf_alaska.apply(
    lambda row: find_nearest(row['lat'], row['lon'], row['year']), 
    axis=1)

merge_df = pd.merge(sub_wp_gulf_alaska,sub_ocean_data_df,on='ocean_data_id', how='left', suffixes=('_wp', '_ocean'))

merge_df.head()

,haul_id,stratum,lat_wp,lon_wp,depth,year_wp,wtcpue,ocean_data_id,time,lat_ocean,lon_ocean,salinity_bottom,water_temp_bottom,year_ocean
0,023-199601-003,210,52.59615,-169.4421,235,1996,9.0793,98709,1996-08-01 09:00:00,52.840000,-169.280029,32.446,6.518000,1996
1,023-199601-004,111,52.78445,-168.7133,109,1996,0.3296,192719,1996-09-01 09:00:00,52.880001,-168.479980,33.532,4.744999,1996
2,023-199601-005,10,52.85188,-168.6160,95,1996,646.7679,192719,1996-09-01 09:00:00,52.880001,-168.479980,33.532,4.744999,1996
3,023-199601-006,10,52.98299,-168.2722,106,1996,256.1357,6131,1996-07-01 09:00:00,52.919998,-168.320007,33.006,4.915999,1996
4,023-199601-007,210,52.96844,-167.5347,227,1996,3.0989,7077,1996-07-01 09:00:00,53.000000,-167.839966,33.822,4.574999,1996


Boosting: 
    - predict biomass on 2019 
    - using lat / long / depth / salinity / temperature 
    
Assignments:

    - Eli : FAQ / Sans
    - Tommy : aggregate data / join frames 
    - Will : model fit / predict 
    - Alonso : gathered data 
    - Sans : math